In [1]:
import requests
import json


def get_ltp_from_api(strike: int, option_type: str):
    """
    Fetches the last traded price for a given option strike and type from the local API.

    Args:
        strike (int): The strike price of the option.
        option_type (str): The type of the option ('CE' or 'PE').

    Returns:
        dict: The parsed JSON response, or None if an error occurs.
    """
    # These parameters can be modified or passed as arguments if needed
    from_date = '17-06-2025'
    to_date = '18-06-2025'
    segment = 'OPTIDX'
    symbol = 'NIFTY'
    year = '2025'
    expiry_date = '26-Jun-2025'
    
    # The API endpoint URL constructed from the function arguments
    api_url = f'http://localhost:8000/api/v1_0/search-data/{from_date}/{to_date}/{segment}/{symbol}/{year}/{expiry_date}/{option_type}/{strike}'

    try:
        # Send a GET request to the specified URL
        api_response = requests.get(api_url)
        
        # Raise an HTTPError if the HTTP request returned an unsuccessful status code
        api_response.raise_for_status()
        
        # Parse the JSON response from the API
        response_data = api_response.json()
        
        print(f"API Response for {strike} {option_type}: {response_data}")
        return response_data

    except requests.exceptions.ConnectionError:
        print(f"Connection Error: Could not connect to the API at {api_url}. Please ensure the local server is running.")
    except requests.exceptions.HTTPError as http_err:
        print(f"HTTP Error occurred: {http_err}")
        if 'api_response' in locals():
            print(f"Response content: {api_response.text}")
    except requests.exceptions.RequestException as err:
        print(f"An unexpected error occurred: {err}")
    except ValueError:
        print("Error: Could not parse JSON response.")
        if 'api_response' in locals():
            print(f"Response content: {api_response.text}")
            
    return None

# Example usage:
response_data = get_ltp_from_api(strike=24900, option_type='CE')
if response_data is not None:
    # Extract and print the last traded price
    last_trade_price = response_data["data"][0]["FH_LAST_TRADED_PRICE"]
    print(f"Last Traded Price: {last_trade_price}")
else:
    print("Failed to fetch the last traded price.")

Connection Error: Could not connect to the API at http://localhost:8000/api/v1_0/search-data/17-06-2025/18-06-2025/OPTIDX/NIFTY/2025/26-Jun-2025/CE/24900. Please ensure the local server is running.
Failed to fetch the last traded price.


In [81]:

def get_ltp_from_api(strike: int, option_type: str):
    """
    Fetches the last traded price for a given option strike and type from the local API.

    Args:
        strike (int): The strike price of the option.
        option_type (str): The type of the option ('CE' or 'PE').

    Returns:
        float: The premium (last traded price), or None if an error occurs.
    """
    from_date = '17-06-2025'
    to_date = '18-06-2025'
    segment = 'OPTIDX'
    symbol = 'NIFTY'
    year = '2025'
    expiry_date = '26-Jun-2025'
    
    api_url = f'http://localhost:8000/api/v1_0/search-data/{from_date}/{to_date}/{segment}/{symbol}/{year}/{expiry_date}/{option_type}/{strike}'

    try:
        api_response = requests.get(api_url)
        api_response.raise_for_status()
        response_data = api_response.json()
        return float(response_data["data"][0]["FH_LAST_TRADED_PRICE"])
    except Exception as e:
        print(f"Error fetching premium for {strike} {option_type}: {e}")
        return None

def calculate_breakeven(buy_ce_strike: int, sell_pe_strike: int):
    """
    Calculates the breakeven points for a simple strategy with one Buy CE and one Sell PE.

    Args:
        buy_ce_strike (int): Strike price of the Call option being bought.
        sell_pe_strike (int): Strike price of the Put option being sold.

    Returns:
        tuple: Breakeven points (lower breakeven, upper breakeven).
    """
    # Fetch premiums using the API
    buy_ce_premium = get_ltp_from_api(buy_ce_strike, "CE")
    sell_pe_premium = get_ltp_from_api(sell_pe_strike, "PE")

    if buy_ce_premium is None or sell_pe_premium is None:
        print("Error: Could not fetch premiums for the given strikes.")
        return None

    # Calculate breakeven points
    lower_breakeven = sell_pe_strike - sell_pe_premium
    upper_breakeven = buy_ce_strike + buy_ce_premium

    return lower_breakeven, upper_breakeven


# Example usage:
buy_ce_strike = 23000
sell_pe_strike = 22000

breakeven_points = calculate_breakeven(buy_ce_strike, sell_pe_strike)
if breakeven_points:
    print(f"Lower Breakeven: {breakeven_points[0]}")
    print(f"Upper Breakeven: {breakeven_points[1]}")

Lower Breakeven: 21996.35
Upper Breakeven: 24868.5


In [118]:
pip install  tabulate


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: C:\Users\Zubrist\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [120]:
import pandas as pd
from tabulate import tabulate
from fyers_apiv3 import fyersModel
# Initialize the Fyers API client
access_token = "eyJhbGciOiJIUzI1NiIsInR5cCI6IkpXVCJ9.eyJhdWQiOlsiZDoxIiwiZDoyIiwieDowIiwieDoxIiwieDoyIl0sImF0X2hhc2giOiJnQUFBQUFCb1V1b2FMYUxMZmJoT0RDd2tIUWFfZnhybkxpN2lVS0xkSGp6eTF0RGNmUnpyMGJOUDExM0xTNURnVHg0cEpDYjg0bllMVThOa1FWaVJZVWdScFIzQmRrcURTRS1WV2VTa2N4Q0g4aExPd0hia3dfRT0iLCJkaXNwbGF5X25hbWUiOiIiLCJvbXMiOiJLMSIsImhzbV9rZXkiOiIzNzJmNjg5NTlmYWQ2NDBkOGEyMmQ3NTEzMWU3ODk0ZjE3MDViYWU5MzNkNjYzMzE3MjY5NjNmZiIsImlzRGRwaUVuYWJsZWQiOiJOIiwiaXNNdGZFbmFibGVkIjoiTiIsImZ5X2lkIjoiWUE0NzM3MyIsImFwcFR5cGUiOjEwMCwiZXhwIjoxNzUwMjkzMDAwLCJpYXQiOjE3NTAyNjQzNDYsImlzcyI6ImFwaS5meWVycy5pbiIsIm5iZiI6MTc1MDI2NDM0Niwic3ViIjoiYWNjZXNzX3Rva2VuIn0.ys0NP0gYo8dYj2cr8E0eOuEP5gtE2YP-EdatQFQ6P6k"  # or your existing token
client_id = "FYZT8L00T9-100"
fyers = fyersModel.FyersModel(client_id=client_id, token=access_token, is_async=False, log_path="")

data = {
    "symbol":"NSE:NIFTY50-INDEX",
    "strikecount":10,
    "timestamp": ""
}

response = fyers.optionchain(data=data)
if response.get('s') == 'ok':
    #print("Option Chain Data:", response['data'])
    option_chain_data = response['data'].get('optionsChain', [])
    
    # Convert to DataFrame
    df = pd.DataFrame(option_chain_data)
    
    # Filter relevant columns for display
    display_columns = ['symbol', 'strike_price', 'option_type', 'ltp', 'oi', 'volume', 'bid', 'ask', 'ltpch', 'ltpchp', 'oich', 'oichp']
    df_display = df[display_columns].fillna('')  # handle missing columns if any

    # Print as table
    print(tabulate(df_display, headers='keys', tablefmt='grid', showindex=False))

else:
    print("Error:",response)

DEBUG:FyersAPIRequest:{'Status Code': 200, 'API': '/options-chain-v3'}


+-----------------------+----------------+---------------+----------+------------+-------------+--------+--------+---------+----------+------------+---------+
| symbol                |   strike_price | option_type   |      ltp | oi         | volume      |    bid |    ask |   ltpch |   ltpchp | oich       | oichp   |
+=======================+================+===============+==========+============+=============+========+========+=========+==========+============+=========+
| NSE:NIFTY50-INDEX     |             -1 |               | 24812    |            |             |   0    |   0    |  -41.35 |    -0.17 |            |         |
+-----------------------+----------------+---------------+----------+------------+-------------+--------+--------+---------+----------+------------+---------+
| NSE:NIFTY2561924300PE |          24300 | PE            |     6.8  | 4282050.0  | 71478300.0  |   6.8  |   7    |   -8.1  |   -54.36 | 310500.0   | 7.82    |
+-----------------------+----------------+----

In [106]:
from fyers_apiv3 import fyersModel

# Initialize the Fyers API client
access_token = "eyJhbGciOiJIUzI1NiIsInR5cCI6IkpXVCJ9.eyJhdWQiOlsiZDoxIiwiZDoyIiwieDowIiwieDoxIiwieDoyIl0sImF0X2hhc2giOiJnQUFBQUFCb1V1b2FMYUxMZmJoT0RDd2tIUWFfZnhybkxpN2lVS0xkSGp6eTF0RGNmUnpyMGJOUDExM0xTNURnVHg0cEpDYjg0bllMVThOa1FWaVJZVWdScFIzQmRrcURTRS1WV2VTa2N4Q0g4aExPd0hia3dfRT0iLCJkaXNwbGF5X25hbWUiOiIiLCJvbXMiOiJLMSIsImhzbV9rZXkiOiIzNzJmNjg5NTlmYWQ2NDBkOGEyMmQ3NTEzMWU3ODk0ZjE3MDViYWU5MzNkNjYzMzE3MjY5NjNmZiIsImlzRGRwaUVuYWJsZWQiOiJOIiwiaXNNdGZFbmFibGVkIjoiTiIsImZ5X2lkIjoiWUE0NzM3MyIsImFwcFR5cGUiOjEwMCwiZXhwIjoxNzUwMjkzMDAwLCJpYXQiOjE3NTAyNjQzNDYsImlzcyI6ImFwaS5meWVycy5pbiIsIm5iZiI6MTc1MDI2NDM0Niwic3ViIjoiYWNjZXNzX3Rva2VuIn0.ys0NP0gYo8dYj2cr8E0eOuEP5gtE2YP-EdatQFQ6P6k"  # or your existing token
client_id = "FYZT8L00T9-100"
fyers = fyersModel.FyersModel(client_id=client_id, token=access_token, is_async=False, log_path="")


data = {
    "symbol":"NSE:NIFTY50-INDEX",
    "ohlcv_flag":"1"
}

response = fyers.depth(data=data)
print(response)

DEBUG:FyersAPIRequest:{'Status Code': 200, 'API': '/depth'}


{'d': {'NSE:NIFTY50-INDEX': {'totalbuyqty': 0, 'totalsellqty': 0, 'ask': [], 'bids': [], 'o': 24788.35, 'h': 24947.55, 'l': 24750.45, 'c': 24853.4, 'chp': -0.17, 'tick_Size': 0.05, 'ch': -41.35, 'ltq': 0, 'ltt': 0, 'ltp': 24812.05, 'v': 0, 'atp': 0, 'lower_ckt': 0, 'upper_ckt': 0, 'expiry': '', 'oi': 0, 'oiflag': False, 'pdoi': 0, 'oipercent': 0}}, 'message': 'Success', 's': 'ok'}


In [114]:
import requests
import json

def fetch_nifty_option_chain(expiry_date: str):
    url = 'https://www.nseindia.com/api/option-chain-v3'

    params = {
        'type': 'Indices',
        'symbol': 'NIFTY',
        'expiry': expiry_date
    }

    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 '
                      '(KHTML, like Gecko) Chrome/126.0.0.0 Safari/537.36',
        'Accept-Language': 'en-US,en;q=0.9',
        'Accept-Encoding': 'gzip, deflate, br',
        'Accept': 'application/json',
        'Referer': 'https://www.nseindia.com/option-chain',
        'Connection': 'keep-alive'
    }

    session = requests.Session()
    session.headers.update(headers)

    try:
        # NSE sets cookies on the home page
        session.get('https://www.nseindia.com', timeout=5)

        response = session.get(url, params=params, timeout=10)
        response.raise_for_status()

        if response.text.strip():  # Check if the response is not empty
            data = response.json()
            return data.get('records', {}).get('data', [])
        else:
            print("Error: Empty response received from the server.")
            return []
    except requests.exceptions.RequestException as e:
        print(f"Request failed: {e}")
        return []
    except json.JSONDecodeError:
        print("Error: Failed to decode JSON response.")
        if 'response' in locals():
            print(f"Response content: {response.text}")
        return []

if __name__ == '__main__':
    expiry = '26-Jun-2025'
    option_chain = fetch_nifty_option_chain(expiry)
    print(option_chain)

    # if option_chain:
    #     for row in option_chain:
    #         print(f"Strike: {row.get('strikePrice')}")
    #         if 'CE' in row:
    #             ce = row['CE']
    #             print(f"  CE - LTP: {ce.get('lastPrice')}, OI: {ce.get('openInterest')}, Vol: {ce.get('totalTradedVolume')}")
    #         if 'PE' in row:
    #             pe = row['PE']
    #             print(f"  PE - LTP: {pe.get('lastPrice')}, OI: {pe.get('openInterest')}, Vol: {pe.get('totalTradedVolume')}")
    #         print('-' * 50)
    # else:
    #     print("No data available for the given expiry date.")

Request failed: 401 Client Error: Unauthorized for url: https://www.nseindia.com/api/option-chain-v3?type=Indices&symbol=NIFTY&expiry=26-Jun-2025
[]
